In [71]:
import pandas as pd

rating_data= pd.read_csv('C:/Jupyter_project/datasets/movie_lens/rating.csv')
row_movie_data= pd.read_csv('C:\Jupyter_project\datasets\movie_lens/movie.csv')

In [90]:
"""
parameters : 
    input_data (DataFrame)
        : movie.csv에서 읽은 DataFrame.
          입력된 DataFrame 객체의 원형은 바뀌지 않습니다.
        
    year (integer)
        : year 미만 영화는 거릅니다.
        
    genres (array of string)
        : 분류된 모든 장르가 genres배열에 포함되지 않는 영화는 거릅니다.


return (DataFrame): 
    year 컬럼이 추가되고 조건에 맞게 필터링된 DataFrame
    

*
title이 (year)로 끝나지 않는 15개의 영화는 아예 제외했습니다.
(ex. movield 7789)
       
"""

def movie_filter(input_data, year=1950, genres=[]):
    data = input_data.copy() 
    
    data['year'] = data['title'].str[-5:-1]
    data['year'] = pd.to_numeric(data['year'], downcast='integer', errors='coerce')
    data = data.dropna(axis=0)    
    
    data['genres'] = data['genres'].str.split('|')
    
    data = data[data['year']>=year]
    
    mask = []
    for genre_ in data['genres']:
        mask.append(True if np.intersect1d(genre_, genres).size > 0 else False) 
            
    data = data[mask]
    
    return data

In [91]:
import numpy as np
movie_data = movie_filter(row_movie_data, genres=['Action', 'Crime'])

In [93]:
rating_data['userId'].value_counts().shape
x= rating_data['userId'].value_counts()>100
y=x[x].index
rating_data = rating_data[rating_data['userId'].isin(y)]

In [94]:
#구한 영화 data만 있는 rating_data를 만든다.

def rating_filter(input_rating, input_movie):
    check  = np.in1d(input_rating['movieId'], input_movie['movieId'])
    for i in check :
        if (check[i] == False).all:
            rating_data = input_rating.drop([i])
    return rating_data

In [95]:
rating_datas= pd.merge(rating_data, movie_data, on = 'movieId', how = 'right')

In [96]:
from scipy.sparse.linalg import svds
def make_prediction_df():
    R_df = rating_datas.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)

    #R은 pivot_table을 numpy matrix로 만든 것
    R = R_df.values
    
    #user_rating_mean은 사용자의 평균 평점
    user_ratings_mean = np.mean(R, axis = 1)
    
    #R_demeaned : 사용자-영화 테이블에 대해 사용자 평균 평점을 뺀 것
    R_demeaned = R - user_ratings_mean.reshape(-1, 1)
        
    #U 행렬, sigma 행렬, V전치행렬을 반환.
    #이때 spicy에 있는 svd를 이용한다.
    U, sigma, Vt = svds(R_demeaned, k = 50)
    
    #sigma는 0이 포함되지 않은 값으로만 구성되어 있다.
    #sigma를 0이 포함된 대칭 행렬로 변환한다.
    sigma = np.diag(sigma)
    
    #SVD가 적용되어 분해된 R_demeaned를 원본 행렬로 복구
    #구한 원본 행렬에 사용자 평균 rating을 더해준다.
    all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1,1)
    
    preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)
    
    return preds_df


In [97]:
def recommend_movies(predictions_df, userID, movies_df, original_ratings_df, num_recommendations=5):
    
    #인덱스로 변환해주어야 해서 -1 
    user_row_number = userID - 1
    
    #앞에서 만든 prediction행렬을 사용자 인덱스에 따라 영화 정렬 -> 영화 평점이 높은 순으로 정렬
    sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(ascending=False)
    
    #원본 rating_data에서 user_id에 해당하는 데이터를 뽑아낸다.
    user_data = original_ratings_df[original_ratings_df.userId == (userID)]
    
    #user_data와 원본 영화 데이터를 합친다.
    user_full = (user_data.merge(movies_df, how = 'left', left_on = 'movieId', right_on = 'movieId').
                     sort_values(['rating'], ascending=False)
                 )
    
    #원본 영화 데이터에서 사용자가 본 영화를 제외한 데이터 추출
    #.merge로 sorted_user_predcitions와 user_full데이터를 합친다.
    #.rename으로 컬럼 이름을 바꾸고 정렬한다
    recommendations = (movies_df[~movies_df['movieId'].isin(user_full['movieId'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movieId',
               right_on = 'movieId').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False)
                      )
                       
    return recommendations


In [98]:
#make_prediction_df()함수 호출하기 전에 input을 넣어줍니다.
preds_df = make_prediction_df()

In [99]:
#24번 사용자가 본 영화 데이터를 토대로 영화 추천 받은 목록
prediction= recommend_movies(preds_df,24, movie_data, rating_data, 10)

In [100]:
prediction.head(10)

,movieId,title,genres,year,Predictions
140,858,"Godfather, The (1972)","[Crime, Drama]",1972.0,4.955399
189,1221,"Godfather: Part II, The (1974)","[Crime, Drama]",1974.0,4.774361
20,111,Taxi Driver (1976),"[Crime, Drama, Thriller]",1976.0,3.082877
653,4262,Scarface (1983),"[Action, Crime, Drama]",1983.0,1.696186
11,70,From Dusk Till Dawn (1996),"[Action, Comedy, Horror, Thriller]",1996.0,1.554705
80,474,In the Line of Fire (1993),"[Action, Thriller]",1993.0,1.489349
85,493,Menace II Society (1993),"[Action, Crime, Drama]",1993.0,1.147451
23,145,Bad Boys (1995),"[Action, Comedy, Crime, Drama, Thriller]",1995.0,1.095014
499,3256,Patriot Games (1992),"[Action, Thriller]",1992.0,1.001665
310,1917,Armageddon (1998),"[Action, Romance, Sci-Fi, Thriller]",1998.0,0.946215
